In [13]:
import os
import shutil
import pickle
import numpy as np
import cv2
from datetime import date
import sklearn

In [20]:
dir = "../data/tracked_fish_3_camera_angles"
destination_dir = "../data/fish_tracking"
video_dir = "tracking_vids"
label_dir = "h5_tracking_files"
camera_angles = ["A","B","C"]

#classes = ["0", "1"]

today = date.today()
today = today.strftime('%Y_%m_%d')
img_list = []

dest_img_path = os.path.join(destination_dir, "images")
dest_label_path = os.path.join(destination_dir, "labels")

# Create directories if not exists
if not os.path.exists(destination_dir):
    os.makedirs(destination_dir)
    os.makedirs(dest_img_path)
    os.makedirs(dest_label_path)

# Loop over all subdir
for d in sorted(os.listdir(dir)):

    # Define paths
    sub_d = os.path.join(dir, d)
    video_path = os.path.join(sub_d, video_dir)
    annotation_path = os.path.join(sub_d, label_dir)
    
    # loop over different angles
    for angle in camera_angles:

        # Load annotation file
        annotation_file = os.path.join(annotation_path, f"{d}Corr{angle}.pkl")
        annotation_dict = pickle.load(open(annotation_file, "rb"))
        
        # Load video
        video_file = os.path.join(video_path, f"{d}_{angle}.mp4")
        cap = cv2.VideoCapture(video_file)
        
        # Loop over frames with annotation.
        # TODO: Need to be extended later on for prey
        i = 0
        for x,y in zip(annotation_dict["0"]["X"],annotation_dict["0"]["Y"]):
            
            ret, frame = cap.read()
            
            if ~np.isnan(x) & ~np.isnan(y):
                # save frame
                
                # Center coordinates is a single map_images folder but not directly align with the annotations.  The current approach is 
                center_coordinates = (int(x), int(y))

                # Radius of circle
                radius = 50
                
                # Determine bbox
                xmin = int(x) - radius
                ymin = int(y) - radius
                xmax = int(x) + radius
                ymax = int(y) + radius

                # Get frame size
                width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH ))
                height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT ))
                # fps =  cap.get(cv2.CAP_PROP_FPS)

                # Blue color in BGR
                color = (0, 255, 0)
                
                # Line thickness of 2 px
                thickness = 2

                # Write onto frame (sanity check)
                image = frame
                # image = cv2.circle(image, center_coordinates, radius, color, thickness)
                
                # Naming scheme
                idx = i
                idx = str(idx)
                idx = idx.zfill(5) 
                image_name = f"{today}_moving_camera_{d}_{angle}_{idx}"

                # Add img to list for data splitting
                img_list.append(image_name)
                
                # Write img
                print(image_name)
                cv2.imwrite(f"{dest_img_path}/{image_name}.jpg", image)
                
                # write label
                label = ["predator", "0", xmin, ymin, xmax, ymax, width, height]
                with open(f'{dest_label_path}/{image_name}.txt', 'w') as f:
                    for item in label:
                        f.write("%s " % item)
                
                i+= 1


                

2022_02_08
2022_02_08_moving_camera_131_A_00000
2022_02_08_moving_camera_131_A_00001
2022_02_08_moving_camera_131_A_00002
2022_02_08_moving_camera_131_A_00003
2022_02_08_moving_camera_131_A_00004
2022_02_08_moving_camera_131_A_00005
2022_02_08_moving_camera_131_A_00006
2022_02_08_moving_camera_131_A_00007
2022_02_08_moving_camera_131_A_00008
2022_02_08_moving_camera_131_A_00009
2022_02_08_moving_camera_131_A_00010
2022_02_08_moving_camera_131_A_00011
2022_02_08_moving_camera_131_A_00012
2022_02_08_moving_camera_131_A_00013
2022_02_08_moving_camera_131_A_00014
2022_02_08_moving_camera_131_A_00015
2022_02_08_moving_camera_131_A_00016
2022_02_08_moving_camera_131_A_00017
2022_02_08_moving_camera_131_A_00018
2022_02_08_moving_camera_131_A_00019
2022_02_08_moving_camera_131_A_00020
2022_02_08_moving_camera_131_A_00021
2022_02_08_moving_camera_131_A_00022
2022_02_08_moving_camera_131_A_00023
2022_02_08_moving_camera_131_A_00024
2022_02_08_moving_camera_131_A_00025
2022_02_08_moving_camera_13

In [21]:

from sklearn.model_selection import train_test_split
data_dir = "../data/fish_tracking"
train, test = train_test_split(img_list, test_size=0.2, random_state=1)

with open(f'{data_dir}/train.txt', 'w') as f:
    for item in sorted(train):
        f.write("%s\n" % item)


with open(f'{data_dir}/test.txt', 'w') as f:
    for item in sorted(test):
        f.write("%s\n" % item)

2434